In [3]:


from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import os
import base64
import email

SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

def authenticate_gmail():
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        from google_auth_oauthlib.flow import InstalledAppFlow
        flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
        creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    return build("gmail", "v1", credentials=creds)



In [4]:
def search_emails(service, email_address, word, after_date):
    query = f"from:{email_address} subject:{word} has:attachment after:{after_date}"
    results = service.users().messages().list(userId="me", q=query).execute()
    messages = results.get("messages", [])
    return messages


In [5]:
def download_attachments(service, messages, save_dir="attachments"):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for msg in messages:
        msg_id = msg["id"]
        message = service.users().messages().get(userId="me", id=msg_id).execute()
        payload = message["payload"]

        if "parts" in payload:
            for part in payload["parts"]:
                if part["filename"]:  # Check if it's an attachment
                    attachment_id = part["body"]["attachmentId"]
                    attachment = service.users().messages().attachments().get(
                        userId="me", messageId=msg_id, id=attachment_id
                    ).execute()

                    file_data = base64.urlsafe_b64decode(attachment["data"].encode("UTF-8"))
                    file_path = os.path.join(save_dir, part["filename"])

                    with open(file_path, "wb") as f:
                        f.write(file_data)
                        print(f"Downloaded: {file_path}")

In [6]:
def search_emails_bysender(service, email_address, word):
    query = f"from:{email_address} subject:{word} has:attachment"
    results = service.users().messages().list(userId="me", q=query).execute()
    messages = results.get("messages", [])
    return messages
def download_attachments_sender(service, messages, save_dir="attachments"):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for msg in messages:
        msg_id = msg["id"]
        message = service.users().messages().get(userId="me", id=msg_id).execute()
        payload = message["payload"]

        if "parts" in payload:
            for part in payload["parts"]:
                if part["filename"]:  # Check if it's an attachment
                    attachment_id = part["body"]["attachmentId"]
                    attachment = service.users().messages().attachments().get(
                        userId="me", messageId=msg_id, id=attachment_id
                    ).execute()

                    file_data = base64.urlsafe_b64decode(attachment["data"].encode("UTF-8"))
                    file_path = os.path.join(save_dir, part["filename"])

                    with open(file_path, "wb") as f:
                        f.write(file_data)
                        print(f"Downloaded: {file_path}")


In [8]:
# if __name__ == "__main__":
#     gmail_service = authenticate_gmail()
#     keyword = "elecf"  # Change to the word you want to filter by
#     messages = search_emails(gmail_service, keyword)

#     if messages:
#         print(f"Found {len(messages)} email(s) with '{keyword}' in subject.")
#         download_attachments(gmail_service, messages)
#     else:
#         print("No emails found.")



In [18]:
# if __name__ == "__main__":
#     gmail_service = authenticate_gmail()

#     sender_email = "nh_kadri@esi.dz"  # Change this to the sender's email
#     keyword = ""  # Change this to the keyword in the subject

#     messages = search_emails_bysender(gmail_service, sender_email, keyword)

#     if messages:
#         print(f"Found {len(messages)} email(s) from {sender_email} with '{keyword}' in the subject.")
#         download_attachments_sender(gmail_service, messages)
#     else:
#         print("No matching emails found.")
if __name__ == "__main__":
    gmail_service = authenticate_gmail()
# nh_kadri@esi.dz
# my_haouas@esi.dz
# b_khelouat@esi.dz 
# w_hidouci@esi.dz
# aa_chader@esi.dz

    sender_email = "aa_chader@esi.dz"  # Change to the sender's email
    keyword = ""  # Change to the keyword in the subject
    after_date = "2025/02/01"  # Change to the desired date (YYYY/MM/DD)

    messages = search_emails(gmail_service, sender_email, keyword, after_date)

    if messages:
        print(f"Found {len(messages)} email(s) from {sender_email} with '{keyword}' in the subject after {after_date}.")
        download_attachments_sender(gmail_service, messages)
    else:
        print("No matching emails found.")



Found 4 email(s) from aa_chader@esi.dz with '' in the subject after 2025/02/01.
Downloaded: attachments\EmpRecords.txt
Downloaded: attachments\Consultations.txt
Downloaded: attachments\Int1-Corrigé.pdf
Downloaded: attachments\TP1_ALSDD_G8-12-13.pdf
Downloaded: attachments\LINKED LISTS.pdf
Downloaded: attachments\TD1_ARRAYS.pdf
